GRUPO 5 - INTEGRANTES:
- Juan Jano Atiquipa Janampa
- Kimberly Dayanne Vigo Miranda
- Andres Alexander Villacorta Barrera
- Luis Enrique Abanto Villar

Link de DataSet: https://www.kaggle.com/datasets/girardi69/marathon-time-predictions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Diccionario de encabezados:

- **id** = Identificador del corredor
- **Marathon** = Carrera realizada
- **Name** = Nombre
- **Category** = Categoría a la que pertenece el corredor
- **km4week** = Promedio de kilometros recorrridos en las 4 últimas semanas
- **sp4week** = Promedio de velocidad en las 4 últimas semanas
- **Crosstraining** = Si el corredor además de correr, horas de práctica de crosstraining
- **Wall21** = Tiempo en completar media maratón
- **MarathonTime** = Tiempo en completar toda la maratín
- **CATEGORY** = Categoría del tiempo empleado para completar la maratón

# 1. Limpieza de datos

### 1.1 Missing values

In [ ]:
# Importamos los dataset o conjunto de datos

data = pd.read_csv('MarathonData.csv')
data.head()

In [ ]:
# Se revisa con qué tipo de datos se va a trabajar

data.info()

In [ ]:
# Se cambia 'Wall21' de objeto a numérico

data['Wall21'] = pd.to_numeric(data['Wall21'], errors = 'coerce')

In [ ]:
# Función para detectar columnas con nulos y su porcentaje

def porcentaje(columna):
  total_null = columna.isnull().sum()
  len_data = len(data)
  return (total_null/len_data)*100

def main(my_data):
  columns_null = [var for var in my_data.columns if my_data[var].isnull().sum() > 0]
  print(columns_null)

  for column_name in columns_null: 
    print(column_name, porcentaje(my_data[column_name]))

main(data)

### 1.2 Imputación

**Nota:** A pesar de que los missing values en 'CrossTraining' superan el 80% no se eliminaron ya que los vacíos representan que el corredor no realiza un entrenamiento adicional.

In [ ]:
# Se llena los vacíos con 0 para aquellas que no practican CrossTraining

data['CrossTraining'] = data['CrossTraining'].fillna(0)
data

In [ ]:
# Eliminamos 6 filas que contienen nulos en 'Category' y 'Wall21'
# Porque no es recomendable imputar la media total a los participantes
# Ya que cada uno pertence a categorías diferentes

data = data.dropna(how = 'any')
data

### 1.3 Outliers

**Nota:** En el caso de la colummna 'km4week' usamos boxplot porque permite visualizar mejor los outliers.

In [ ]:
data.boxplot(column = ['km4week'])
plt.show()

In [ ]:
sns.scatterplot(x='id', y='sp4week', data=data)
plt.show()

In [ ]:
sns.scatterplot(x='id', y='Wall21', data=data)
plt.show()

In [ ]:
sns.scatterplot(x='id', y='MarathonTime', data=data)
plt.show()

In [ ]:
!pip install feature_engine

In [ ]:
# Se usa el método de cappers para los outliers

from feature_engine.outliers import ArbitraryOutlierCapper

In [ ]:
data.km4week.max(), data.sp4week.max()

In [ ]:
# Se establece límites de manera arbitraria

capper = ArbitraryOutlierCapper(max_capping_dict={
    'km4week': 122, 'sp4week': 15},
    min_capping_dict={
    'km4week': 17, 'sp4week': 9})

capper.fit(data.fillna(0))

In [ ]:
capper.right_tail_caps_

In [ ]:
capper.left_tail_caps_

In [ ]:
data = capper.transform(data.fillna(0))
data.km4week.min(), data.sp4week.min()

In [ ]:
data.km4week.max(), data.sp4week.max()

In [ ]:
# Comprobamos que ya no existen outliers

data.boxplot(column = ['km4week'])
plt.show()

# 2. Transformación y selección de datos
### 2.1 Label Encoding

In [ ]:
# Revisamos las etiquetas de la columna 'CrossTraining'

data['CrossTraining'].unique()

In [ ]:
# Damos un número a cada etiqueta

val_cross = {
    'CrossTraining': {
        'ciclista 1h': 1,
        'ciclista 3h': 2,
        'ciclista 4h': 3,
        'ciclista 5h': 4,
        'ciclista 13h': 5
    }
}

data.replace(val_cross, inplace = True)
data

In [ ]:
# Confirmamos que se haya realizado el cambio

data['CrossTraining'].unique()

In [ ]:
# Revisamos las etiquetas de la columna 'Category'

data['Category'].unique()

In [ ]:
# Damos un número a cada etiqueta

val_cat = {
    'Category': {
        'MAM': 1,
        'M40': 2,
        'M45': 3,
        'M50': 4,
        'M55': 5,
        'WAM': 6,
    }
}

data.replace(val_cat, inplace = True)
data

In [ ]:
# Confirmamos que se haya realizado el cambio

data['Category'].unique()

### 2.2 Selección de datos

In [ ]:
# eliminamos las columnas que no aportan al modelo

data = data.drop(columns = ['Name'])
data = data.drop(columns = ['id'])
data = data.drop(columns = ['Marathon'])
data = data.drop(columns = ['CATEGORY'])
data 

# 3. Escalado de datos continuos con output continuo

In [ ]:
# Importamos las librerias de estadistica
from sklearn.model_selection import train_test_split

# Importamos las librerias para el feature selection
from sklearn.preprocessing import StandardScaler  # Normalización

In [ ]:
data['MarathonTime'].plot.hist()
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('MarathonTime', axis=1),
                                                    data['MarathonTime'],
                                                    test_size=0.3,
                                                    random_state=0)

X_train.shape, X_test.shape

In [ ]:
means = X_train.mean(axis=0)
means

In [ ]:
ranges = X_train.max(axis=0)-X_train.min(axis=0)
ranges

In [ ]:
X_train_scaled = (X_train - means) / ranges
X_test_scaled = (X_test - means) / ranges

In [ ]:
# Comparamos la distribución de las variables antes y después del escalado

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

# Antes
ax1.set_title('Antes del escalado')
sns.kdeplot(X_train['km4week'], ax=ax1)
sns.kdeplot(X_train['sp4week'], ax=ax1)
sns.kdeplot(X_train['Wall21'], ax=ax1)

# Despues
ax2.set_title('Despues de la normalizacion')
sns.kdeplot(X_train_scaled['km4week'], ax=ax2)
sns.kdeplot(X_train_scaled['sp4week'], ax=ax2)
sns.kdeplot(X_train_scaled['Wall21'], ax=ax2)
plt.show()

In [ ]:
np.round(X_train_scaled.describe(include="all"), 1)

# 4. Entrenamiento de modelo

In [ ]:
# Usamos el modelo de regresión lineal ya que nuestro 'x' e 'y' son numéricos

from sklearn.linear_model import LinearRegression

In [ ]:
modelo = LinearRegression()
modelo.fit(X_train_scaled, y_train)

In [ ]:
predicciones = modelo.predict(X_test_scaled)
predicciones

### Determinamos el error cuadrático medio

In [ ]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(y_test, predicciones))
print('Error porcentual: %f' % (error*100))

### Probando el modelo

In [ ]:
# Ingresamos variable aleatorias para saber cuánto 
# Tiempo se demora en completar la maratón

nuevo_corredor = pd.DataFrame(np.array([[1,100,12,0,1.45]]),columns=['Category', 'km4week','sp4week', 'CrossTraining','Wall21'])
nuevo_corredor

In [ ]:
modelo.predict(nuevo_corredor)